## LLM

In [1]:
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
from src.tokenizer import TokenizerBPE, word_split, normalize_to_ascii, pair_freq
from src.data_handling import read_first_n


import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm
from src.data_handling import fused_qa
import sys
import random
import json

# disable gpu for testing purposes
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [62]:
with open('corpus/triviaqa-rc/qa/web-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_web_list = []
answer_web_list = []
evidence_web_list = []

for qa in tqdm(squad['Data']):
    question_web_list.append(qa['Question'])
    answer_web_list.append(qa['Answer']["Value"])
    sr = qa['SearchResults']
    if len(sr) > 0: 
        evidence_web_list.append(sr[0]["Filename"])
    else:
        evidence_web_list.append(None)
    

  0%|          | 0/76496 [00:00<?, ?it/s]

In [87]:
path = "corpus/triviaqa-rc/evidence/web/"

txt_list = []

for evidence in tqdm(evidence_web_list):
    if evidence is not None:
        txt = open(path + evidence, 'r', encoding='utf-8').read()
        txt_list.append(txt)
    

  0%|          | 0/76496 [00:00<?, ?it/s]

In [88]:
txt = "".join(txt_list)
print("Length of txt: ", len(txt))

Length of txt:  1163223076


In [4]:
with open('corpus/triviaqa-rc/qa/wikipedia-train.json', 'r', encoding='utf-8') as f:
    squad = json.load(f)

question_wiki_list = []
answer_wiki_list = []
evidence_wiki_list = []

for qa in tqdm(squad['Data']):
    question_wiki_list.append(qa['Question'])
    answer_wiki_list.append(qa['Answer']["Value"])
    sr = qa["EntityPages"]
    if len(sr) > 0: 
        evidence_wiki_list.append(sr[0]["Filename"])
    else:
        evidence_wiki_list.append(None)
    

  0%|          | 0/61888 [00:00<?, ?it/s]

In [27]:
path = "corpus/triviaqa-rc/evidence/wikipedia/"

evidence_raw_list = []

seen = set([])

for i, evidence in tqdm(enumerate(evidence_wiki_list)):
    if evidence is not None:
        evidence = evidence.replace(":", "_").replace("?", "_").replace("*", "_").replace('"', "_")
        if not evidence in seen:
            seen.add(evidence)
            try:
                txt = open(path + evidence, 'r', encoding='utf-8').read()
                evidence_raw_list.append(txt)
            except:
                print("Error reading file: ", evidence)
                continue

0it [00:00, ?it/s]

In [32]:
print(len(evidence_raw_list))

28406


In [33]:
text = "".join(evidence_raw_list)
print("Length of question_web_list: ", len(text))

Length of question_web_list:  529632069


In [29]:
text = "".join(question_wiki_list + answer_wiki_list)
print("Length of question_web_list: ", len(text))

Length of question_web_list:  5580683


In [41]:
random.seed(42)

tokenizer = TokenizerBPE(evidence_raw_list[:3000], 16000, lowercase=True)
tokenizer.destroy_hash()

pkl.dump(tokenizer, open('tokenizers/trivia_16k.pkl', 'wb'))

Lowercasing corpus


  0%|          | 0/3000 [00:00<?, ?it/s]

Merging tokens


  0%|          | 0/16000 [00:00<?, ?it/s]

ValueError: 'list' argument must have no negative elements

In [9]:
def fused_qa(question_list, answer_list):
    q ="<q>"
    a = "<a>"
    sos = "<s>"
    eos = "</s>"

    corpus_list = []
    for question, answer in tqdm(list(zip(question_list, answer_list))):
        qa = [sos, q, question, a, answer, eos]
        corpus_list.extend(qa)
    
    return "".join(corpus_list)


def fused_cnn(corpus_list):
    sos = "<s>"
    eos = "</s>"

    corpus_padded_list = []
    for line in corpus_list:
        line = [sos, line, eos]
        corpus_padded_list.extend(line)
    
    return "".join(corpus_padded_list)

In [10]:
tokenizer = pkl.load(open('tokenizers/CNN_QA8k_balanced.pkl', 'rb'))
tokenizer.create_hash()

tokenizer.add_special_tokens(["<s>", "</s>", "<q>", "<a>",])

In [11]:
question_list = [line.lower() for line in question_list]
answer_list = [line.lower() for line in answer_list]

In [12]:
corpus_qa_fused = fused_qa(question_list, answer_list)

corpus_qa_encoded = tokenizer.encode(corpus_qa_fused)

  0%|          | 0/86820 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [13]:
corpus_cnn_fused = fused_cnn(corpus_cnn)

corpus_cnn_encoded = tokenizer.encode(corpus_cnn_fused)

  0%|          | 0/8000 [00:00<?, ?it/s]

In [14]:
corpus = tf.concat([corpus_cnn_encoded, corpus_qa_encoded], axis=1)
pkl.dump(corpus, open('corpus/corpus_CNN_QA8k_balanced', 'wb'))

## 